In [3]:
import scipy
import numpy as np

In [198]:
A_theta = [['cos(T/2)','-sin(T/2)'], ['sin(T/2)', 'cos(T/2)']]
P_alpha =[['exp(-i*A/2)', '0'], ['0', 'exp(i*A/2)']]
P_gamma =[['exp(-i*G/2)', '0'], ['0', 'exp(i*G/2)']]

CNOT_10 = [['1','0', '0', '0'], ['0', '1', '0', '0'], ['0', '0', '0', '1'], ['0', '0', '1', '0']]
CNOT_01 = [['1','0', '0', '0'], ['0', '0', '0', '1'], ['0', '0', '1', '0'], ['0', '1', '0', '0']]

In [226]:
def replace_arg(matrix_list, name : str, val=''):
    if val == '':
        return matrix_list
    dim = len(matrix_list)
    R = []
    for i in range(dim):
        R.append([])
        for j in range(dim):
            R[i].append('0')
    for i, line in enumerate(matrix_list):
        for j, elem in enumerate(line):
            R[i][j] = matrix_list[i][j].replace(name, f'({val})')
    return R

In [230]:
def Aa(theta=''):
    return replace_arg(A_theta, 'T', theta)

def Pa(alpha=''):
    return replace_arg(P_alpha, 'A', alpha)

def Pg(gamma=''):
    return replace_arg(P_gamma, 'G', gamma)

In [231]:
def printWolfrMtrxFmt(lists, n='', elem_delim=''):
    print('{', end='')
    for idx, line in enumerate(lists):
        print('{', end='')
        for elem in line[:-1]:
            print(f"{elem}, ", end=elem_delim)
        print(f"{line[-1]}", end=elem_delim)
        if idx != (len(lists) - 1):
            print('}, ', end=n)
        else:
            print('}', end=n)
    print('}')
    
def printWolfrListFmt(lists, line_delim='\n', elem_delim=''):
    print(f'{len(lists)} matrix lines:')
    print('{', end=line_delim)
    for idx, line in enumerate(lists):
        for elem in line[:-1]:
            print(f"{elem}, ", end=elem_delim)
        print(f"{line[-1]}", end=elem_delim)
        if idx != (len(lists) - 1):
            print(', ', end=line_delim)
        else:
            print(f'{line_delim}}}')

In [232]:
arg = 'pi/3'
printWolfrListFmt(Aa(arg))
printWolfrMtrxFmt(Aa())
printWolfrMtrxFmt(Pa())

2 matrix lines:
{
cos((pi/3)/2), -sin((pi/3)/2), 
sin((pi/3)/2), cos((pi/3)/2)
}
{{cos(T/2), -sin(T/2)}, {sin(T/2), cos(T/2)}}
{{exp(-i*A/2), 0}, {0, exp(i*A/2)}}


In [203]:
def decart2_2(A, B):
    lines = [[], [], [], []]
    for line_idx, line in enumerate(lines):
        if (line_idx <= 1):
            A_line = A[0]
        else:
            A_line = A[1]
            
        B_line = B[line_idx % 2]
            
        for i in range(4):
            line.append(f'({A_line[i // 2]}) * ({B_line[i % 2]})')
    return lines   

# works correct & copress output
def decart2_2_2(A, B):
    lines = [[], [], [], []]
    for line_idx, line in enumerate(lines):
        if (line_idx <= 1):
            A_line = A[0]
        else:
            A_line = A[1]
            
        B_line = B[line_idx % 2]
            
        for i in range(4):
            if (A_line[i // 2] == '0' or B_line[i % 2] == '0'):
                line.append(f'0')
            else:
                line.append(f'({A_line[i // 2]}) * ({B_line[i % 2]})')
    return lines  

In [204]:
A = [['1', '0'], ['0', 'i']]
B = [['0', '-1'], ['1', '0']]

In [205]:
printWolfrListFmt(decart2_2(A, B))
printWolfrListFmt(decart2_2_2(A, B))

4 matrix lines:
{
(1) * (0), (1) * (-1), (0) * (0), (0) * (-1), 
(1) * (1), (1) * (0), (0) * (1), (0) * (0), 
(0) * (0), (0) * (-1), (i) * (0), (i) * (-1), 
(0) * (1), (0) * (0), (i) * (1), (i) * (0)
}
4 matrix lines:
{
0, (1) * (-1), 0, 0, 
(1) * (1), 0, 0, 0, 
0, 0, 0, (i) * (-1), 
0, 0, (i) * (1), 0
}


In [211]:
def matrix_multiplication(M, N):
    dim = len(M)
    # List to store matrix multiplication result
    R = []
    for i in range(dim):
        R.append([])
        for j in range(dim):
            R[i].append('0')
    
    for i in range(0, dim):
        for j in range(0, dim):
            for k in range(0, dim):
                R[i][j] += f" + ( ({M[i][k]}) * ({N[k][j]}) )"
    return R

def matrix_multiplication2(L, R):
    dim_lm = len(L)
    dim_ln = len(L[0])
    dim_rm = len(R)
    dim_rn = len(R[0])
    assert(dim_ln == dim_rm)
    
    
    # List to store matrix multiplication result
    X = []
    for i in range(dim_lm):
        X.append([])
        for j in range(dim_rn):
            X[i].append('0')
    
    for i in range(0, dim_lm):
        for j in range(0, dim_rn):
            for k in range(0, dim_rm):
                X[i][j] += f" + ( ({L[i][k]}) * ({R[k][j]}) )"
    return X

def matrix_multiplication3(L, R):
    dim_lm = len(L)
    dim_ln = len(L[0])
    dim_rm = len(R)
    dim_rn = len(R[0])
    assert(dim_ln == dim_rm)
    
    
    # List to store matrix multiplication result
    X = []
    for i in range(dim_lm):
        X.append([])
        for j in range(dim_rn):
            X[i].append('0')
    
    for i in range(0, dim_lm):
        for j in range(0, dim_rn):
            for k in range(0, dim_rm):
                if (L[i][k] == '0' or R[k][j] == '0'):
                    continue
                X[i][j] += f" + ( ({L[i][k]}) * ({R[k][j]}) )"
    return X

# last version (works correct & compress output)
def matrix_multiplication4(L, R):
    dim_lm = len(L)
    dim_ln = len(L[0])
    dim_rm = len(R)
    dim_rn = len(R[0])
    assert(dim_ln == dim_rm)
    
    
    # List to store matrix multiplication result
    X = []
    for i in range(dim_lm):
        X.append([])
        for j in range(dim_rn):
            X[i].append('')
    
    for i in range(0, dim_lm):
        for j in range(0, dim_rn):
            for k in range(0, dim_rm):
                if (L[i][k] == '0' or R[k][j] == '0' or L[i][k] == '' or R[k][j] == ''):
                    continue
                plus='+'
                if (X[i][j] == ''):
                    plus = ''
                X[i][j] += f"{plus}({L[i][k]})*({R[k][j]})"
    return X

In [212]:
A = [['1', '2'], ['0', '4']]
B = [['-1', '-2'], ['-3', '-4']]
C = [['1'], ['2']]
printWolfrListFmt(matrix_multiplication(A, B))
printWolfrListFmt(matrix_multiplication2(A, B))
printWolfrListFmt(matrix_multiplication3(A, B))
printWolfrListFmt(matrix_multiplication4(A, B))

2 matrix lines:
{
0 + ( (1) * (-1) ) + ( (2) * (-3) ), 0 + ( (1) * (-2) ) + ( (2) * (-4) ), 
0 + ( (0) * (-1) ) + ( (4) * (-3) ), 0 + ( (0) * (-2) ) + ( (4) * (-4) )
}
2 matrix lines:
{
0 + ( (1) * (-1) ) + ( (2) * (-3) ), 0 + ( (1) * (-2) ) + ( (2) * (-4) ), 
0 + ( (0) * (-1) ) + ( (4) * (-3) ), 0 + ( (0) * (-2) ) + ( (4) * (-4) )
}
2 matrix lines:
{
0 + ( (1) * (-1) ) + ( (2) * (-3) ), 0 + ( (1) * (-2) ) + ( (2) * (-4) ), 
0 + ( (4) * (-3) ), 0 + ( (4) * (-4) )
}
2 matrix lines:
{
(1)*(-1)+(2)*(-3), (1)*(-2)+(2)*(-4), 
(4)*(-3), (4)*(-4)
}


In [214]:
def transpose_vector(v : list):
    r = []
    for elem in v[0]:
        r.append([elem])
    return r

In [215]:
a = [['1', '2', '3']]
print(transpose_vector(a))
b = transpose_vector(a)
printWolfrListFmt(matrix_multiplication4(a, b))

[['1'], ['2'], ['3']]
1 matrix lines:
{
(1)*(1)+(2)*(2)+(3)*(3)
}


In [216]:
def reduction4_4_1(M):
    R = [['', ''],['', '']]
    R[0][0] = f'{M[0][0]} + {M[1][1]}'
    R[0][1] = f'{M[0][2]} + {M[1][3]}'
    R[1][0] = f'{M[2][0]} + {M[3][1]}'
    R[1][1] = f'{M[2][2]} + {M[3][3]}'
    return R

def reduction4_4_2(M):
    R = [['', ''],['' ,'']]
    R[0][0] = f'{M[0][0]} + {M[2][2]}'
    R[0][1] = f'{M[0][1]} + {M[2][3]}'
    R[1][0] = f'{M[1][0]} + {M[3][2]}'
    R[1][1] = f'{M[1][1]} + {M[3][3]}'
    return R

# Чистовик
## Задача 2 (вариант б)

In [217]:
theta_1 = 'pi/2'
gamma_1 = 'pi/4'
theta_2 = 'pi'
gamma_2 = 'pi/3'

In [218]:
U1 = decart2_2_2(Pg(gamma_1), Aa(theta=theta_2))
U2 = decart2_2_2(Aa(theta_1), Pg(gamma_2))

U_CNOT10 = matrix_multiplication4(U2, CNOT_10)
U = matrix_multiplication4(U_CNOT10, U1)

In [219]:
xsi0 = [['0'],['1/sqrt(2)'],['-1/sqrt(2)'],['0']]
xsi = matrix_multiplication4(U, xsi0)
ro = matrix_multiplication4(xsi, transpose_vector(xsi))

In [220]:
printWolfrListFmt(Pg(gamma_1))
printWolfrListFmt(Pg(gamma_2))
printWolfrListFmt(Aa(theta_1))
printWolfrListFmt(Aa(theta_2))
print("U 1,2:")
printWolfrListFmt(U1)
printWolfrListFmt(U2)
print("U:")
printWolfrListFmt(U, elem_delim='\n')

2 matrix lines:
{
exp(-i*(pi/4)/2), 0, 
0, exp(i*(pi/4)/2)
}
2 matrix lines:
{
exp(-i*(pi/3)/2), 0, 
0, exp(i*(pi/3)/2)
}
2 matrix lines:
{
cos((pi/2)/2), -sin((pi/2)/2), 
sin((pi/2)/2), cos((pi/2)/2)
}
2 matrix lines:
{
cos((pi)/2), -sin((pi)/2), 
sin((pi)/2), cos((pi)/2)
}
U 1,2:
4 matrix lines:
{
(exp(-i*(pi/4)/2)) * (cos((pi)/2)), (exp(-i*(pi/4)/2)) * (-sin((pi)/2)), 0, 0, 
(exp(-i*(pi/4)/2)) * (sin((pi)/2)), (exp(-i*(pi/4)/2)) * (cos((pi)/2)), 0, 0, 
0, 0, (exp(i*(pi/4)/2)) * (cos((pi)/2)), (exp(i*(pi/4)/2)) * (-sin((pi)/2)), 
0, 0, (exp(i*(pi/4)/2)) * (sin((pi)/2)), (exp(i*(pi/4)/2)) * (cos((pi)/2))
}
4 matrix lines:
{
(cos((pi/2)/2)) * (exp(-i*(pi/3)/2)), 0, (-sin((pi/2)/2)) * (exp(-i*(pi/3)/2)), 0, 
0, (cos((pi/2)/2)) * (exp(i*(pi/3)/2)), 0, (-sin((pi/2)/2)) * (exp(i*(pi/3)/2)), 
(sin((pi/2)/2)) * (exp(-i*(pi/3)/2)), 0, (cos((pi/2)/2)) * (exp(-i*(pi/3)/2)), 0, 
0, (sin((pi/2)/2)) * (exp(i*(pi/3)/2)), 0, (cos((pi/2)/2)) * (exp(i*(pi/3)/2))
}
U:
4 matrix lines:
{
(((cos((pi/2)/2)

In [221]:
printWolfrListFmt(xsi, elem_delim='\n')
printWolfrListFmt(ro)

4 matrix lines:
{
((((cos((pi/2)/2)) * (exp(-i*(pi/3)/2)))*(1))*((exp(-i*(pi/4)/2)) * (-sin((pi)/2))))*(1/sqrt(2))+((((-sin((pi/2)/2)) * (exp(-i*(pi/3)/2)))*(1))*((exp(i*(pi/4)/2)) * (sin((pi)/2))))*(-1/sqrt(2))
, 
((((cos((pi/2)/2)) * (exp(i*(pi/3)/2)))*(1))*((exp(-i*(pi/4)/2)) * (cos((pi)/2))))*(1/sqrt(2))+((((-sin((pi/2)/2)) * (exp(i*(pi/3)/2)))*(1))*((exp(i*(pi/4)/2)) * (cos((pi)/2))))*(-1/sqrt(2))
, 
((((sin((pi/2)/2)) * (exp(-i*(pi/3)/2)))*(1))*((exp(-i*(pi/4)/2)) * (-sin((pi)/2))))*(1/sqrt(2))+((((cos((pi/2)/2)) * (exp(-i*(pi/3)/2)))*(1))*((exp(i*(pi/4)/2)) * (sin((pi)/2))))*(-1/sqrt(2))
, 
((((sin((pi/2)/2)) * (exp(i*(pi/3)/2)))*(1))*((exp(-i*(pi/4)/2)) * (cos((pi)/2))))*(1/sqrt(2))+((((cos((pi/2)/2)) * (exp(i*(pi/3)/2)))*(1))*((exp(i*(pi/4)/2)) * (cos((pi)/2))))*(-1/sqrt(2))

}
4 matrix lines:
{
(((((cos((pi/2)/2)) * (exp(-i*(pi/3)/2)))*(1))*((exp(-i*(pi/4)/2)) * (-sin((pi)/2))))*(1/sqrt(2))+((((-sin((pi/2)/2)) * (exp(-i*(pi/3)/2)))*(1))*((exp(i*(pi/4)/2)) * (sin((pi)/2))))*(-

Тут вручную вбил матрицу, ибо тут, к сожалению, достигается предельная длина символьной последовательности вольфрама(

In [222]:
val1='(-exp(7*i*pi/24)+exp(-i*pi/24))'
val2='(-exp(-7*i*pi/24)-exp(-i*pi/24))'
ro = [[f'{val1}^2', '0', f'{val1}*{val2}', '0'], ['0', '0', '0', '0'], [f'{val1}*{val2}', '0', f'{val2}^2', '0'], ['0', '0', '0', '0']]

ro1 = reduction4_4_1(ro)
ro2 = reduction4_4_2(ro)

printWolfrListFmt(ro1)
printWolfrListFmt(ro2)

2 matrix lines:
{
(-exp(7*i*pi/24)+exp(-i*pi/24))^2 + 0, (-exp(7*i*pi/24)+exp(-i*pi/24))*(-exp(-7*i*pi/24)-exp(-i*pi/24)) + 0, 
(-exp(7*i*pi/24)+exp(-i*pi/24))*(-exp(-7*i*pi/24)-exp(-i*pi/24)) + 0, (-exp(-7*i*pi/24)-exp(-i*pi/24))^2 + 0
}
2 matrix lines:
{
(-exp(7*i*pi/24)+exp(-i*pi/24))^2 + (-exp(-7*i*pi/24)-exp(-i*pi/24))^2, 0 + 0, 
0 + 0, 0 + 0
}


### Ответы сверял с печатной версией решения (вариант б)

## Задача 1

In [233]:
M1 = matrix_multiplication4(Pa(),Aa())
M = matrix_multiplication4(M1, Pg())

printWolfrListFmt(M)

2 matrix lines:
{
((exp(-i*A/2))*(cos(T/2)))*(exp(-i*G/2)), ((exp(-i*A/2))*(-sin(T/2)))*(exp(i*G/2)), 
((exp(i*A/2))*(sin(T/2)))*(exp(-i*G/2)), ((exp(i*A/2))*(cos(T/2)))*(exp(i*G/2))
}
